# Assignment 6 - Hamed Ahmadinia 
1. Use yeast dataset from UCI http://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data
2. Remove the first column and use the last column as the target
3. Only leave CYT and VAC classes
4. Replace [0.3, 0.5, 0.7] in feature 2 to null
5. Replace [0.26, 0.36, 0.64] in feature 3 to null
6. Split the data

7. Impute the data (or not, it's your call)
8. Build a outlier detection model to classify VAC from CYT, i.e. 0 from 1
9. Build a classifer using sample augmentation techniques to flassify VAC from CYT, i.e. 0 from 1
10. Try different methods and hyper paramters

11. Report perfromance using F-1 score

In [27]:
# seting the required environment
import pandas as pd
import numpy as np

In [28]:
# loading the dataset "yeast.data" and seting whitespace to be used as the sep.
# Adiing Prefix to column numbers when no header, e.g. ‘v’ for v0, v1, …
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data', header=None, delim_whitespace=True, prefix='v')

# from the 3th columns of dataframe onwards with all rows
data = data.iloc[:,2:]

# Removing all classeses from dataframe coloumn v9 with clases except "CYT" and "VAC"  
# we used ".reset_index()" to reset the index of the DataFrame, and use the default one instead
# we used "drop('index', axis=1)" to drop labels from ‘index’ and axis=1 along the columns.
data = data.loc[(data.v9 == 'VAC')|(data.v9 == 'CYT'), :].reset_index().drop('index', axis=1)

# replace value in clases "CYT" with 1 and "VAC" with 0 in the related columns which is here "v9" in our dataframe
data=data.replace('CYT', 1)
data=data.replace('VAC', 0)

# show the first 10 rows for the object based on position
data.head(10)

,v2,v3,v4,v5,v6,v7,v8,v9
0,0.40,0.56,0.17,0.5,0.5,0.49,0.22,1
1,0.39,0.60,0.15,0.5,0.0,0.58,0.30,1
2,0.42,0.57,0.35,0.5,0.0,0.53,0.25,1
3,0.44,0.52,0.11,0.5,0.0,0.50,0.22,1
4,0.39,0.50,0.11,0.5,0.0,0.49,0.40,1
5,0.40,0.50,0.16,0.5,0.0,0.50,0.22,1
6,0.44,0.48,0.22,0.5,0.0,0.51,0.22,1
7,0.63,0.42,0.30,0.5,0.0,0.49,0.22,1
8,0.53,0.52,0.13,0.5,0.0,0.55,0.22,1
9,0.53,0.52,0.15,0.5,0.0,0.58,0.22,1


In [29]:
# replacing [0.3, 0.5, 0.7] in feature 2 to null which is here coulum "v2"
# replace [0.26, 0.36, 0.64] in feature 3 to null which is here coulum "v3"
# NaN is short for Not a number. It is used to represent entries that are undefined
data["v2"] = data.v2.replace([0.3, 0.5, 0.7], np.nan)
data["v3"] = data.v3.replace([0.26, 0.36, 0.64], np.nan)

# checking dataframe shape
data.shape

(493, 8)

In [30]:
# Split the dataframe "yeast.data"
X, y = data.iloc[:,:-1], data.iloc[:, -1]

In [31]:
# importing "sklearn.impute.KNNImputer"  in order to imputation for completing missing values using k-Nearest Neighbors.
from sklearn.impute import KNNImputer

# each sample’s missing values are imputed using the mean value from 2 neighbors nearest neighbors found in the training set
# we  used ‘uniform’ as uniform weights in which All points in each neighborhood are weighted equally
impute = KNNImputer(n_neighbors=2, weights='uniform')

# defining X_new as Fit to data, then transform it.
X_new = pd.DataFrame(impute.fit_transform(X), columns=X.columns)

# show the first 10 rows for "X_new" which we defined based on position
X_new.head(10)

,v2,v3,v4,v5,v6,v7,v8
0,0.40,0.56,0.17,0.5,0.5,0.49,0.22
1,0.39,0.60,0.15,0.5,0.0,0.58,0.30
2,0.42,0.57,0.35,0.5,0.0,0.53,0.25
3,0.44,0.52,0.11,0.5,0.0,0.50,0.22
4,0.39,0.50,0.11,0.5,0.0,0.49,0.40
5,0.40,0.50,0.16,0.5,0.0,0.50,0.22
6,0.44,0.48,0.22,0.5,0.0,0.51,0.22
7,0.63,0.42,0.30,0.5,0.0,0.49,0.22
8,0.53,0.52,0.13,0.5,0.0,0.55,0.22
9,0.53,0.52,0.15,0.5,0.0,0.58,0.22


In [32]:
# Build a outlier detection model to classify VAC from CYT, i.e. 0 from 1
# we used sklearn.svm.OneClassSVM as Unsupervised Outlier Detection and estimate the support of a high-dimensional distribution
# Importing neClassSVM from sklearn.svm in which LIBSVM is an integrated software for support vector classification
from sklearn.svm import OneClassSVM

# fit model in OneClassSVM and Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid is set to 'auto' which uses 1 / n_features.
clf = OneClassSVM(gamma='auto').fit(X_new)

# The predict method predicts our actual testing dataset with regard to the fitted (training) data "X_new"
clf.predict(X_new)

array([-1, -1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1, -1, -1,  1,  1,
       -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1,  1,  1,  1,  1,
       -1,  1, -1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1, -1,  1,  1,
        1,  1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1,
        1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1,  1, -1,
       -1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1, -1, -1,  1,  1,  1, -1,
       -1,  1, -1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1,  1, -1,
       -1, -1,  1,  1, -1,  1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,
       -1,  1,  1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1, -1,  1, -1,  1,
        1, -1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1, -1, -1,  1, -1, -1,
       -1, -1,  1, -1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1, -1, -1,  1,
        1, -1,  1, -1,  1,  1, -1, -1,  1,  1,  1, -1, -1, -1,  1,  1,  1,
       -1,  1, -1, -1, -1

In [33]:
# checking series containing counts of unique values.
y.value_counts()

1    463
0     30
Name: v9, dtype: int64

In [34]:
# instaling Imbalanced-learn (imported as imblearn )
# it is a MIT-licensed library relying on scikit-learn (imported as sklearn )
# it provides tools when dealing with classification with imbalanced classes
!pip install -U imbalanced-learn

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [35]:
# importing "train_test_split" packeges for Split arrays or matrices into random train and test subsets
from sklearn.model_selection import train_test_split

# making traning and test sets by spliting data 
xtrain, xtest, ytrain, ytest = train_test_split(X_new, y, test_size=.2)

In [37]:
# impoerting cross_validate for evaluate metric(s) by cross-validation and also record fit/score times
from sklearn.model_selection import cross_validate

# impoerting DummyClassifier as a classifier that makes predictions using simple rules.
# This classifier is useful as a simple baseline to compare with other (real) classifiers.
from sklearn.dummy import DummyClassifier

# Compute the F1 score, also known as balanced F-score or F-measure.
# The F1 score can be interpreted as a harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. 
# The relative contribution of precision and recall to the F1 score are equal
from sklearn.metrics import f1_score

# define "dummy_clf"  with strategy as most_frequent which means always predicts the most frequent label in the training set
dummy_clf = DummyClassifier(strategy="most_frequent")

# We performed a cross-validation evaluation to get an estimate of the test score.
# As a baseline, we could use a classifier which will always predict the majority class independently of the features provided.
scoring = ["accuracy", "balanced_accuracy"]
cv_result = cross_validate(dummy_clf, xtrain, ytrain, scoring=scoring)
print(f"Accuracy score of a dummy classifier is: {cv_result['test_accuracy'].mean():.3f}")
print(
    f"Balanced accuracy score of a dummy classifier is: "
    f"{cv_result['test_balanced_accuracy'].mean():.3f}"
)
dummy_clf.fit(xtrain, ytrain)
print(f1_score(dummy_clf.predict(xtest), ytest))

Accuracy score of a dummy classifier is: 0.934
Balanced accuracy score of a dummy classifier is: 0.500
0.979381443298969


In [12]:
# checking series containing counts of unique values.
ytest.value_counts()

1    96
0     3
Name: v9, dtype: int64

In [40]:
# We used a dictionary and a list to continuously store the results of our experiments and show them as a pandas dataframe
index = []
scores = {"Accuracy": [], "Balanced accuracy": [], "f1_score": []}

In [14]:
# Before to train a real machine learning model, we stored the results obtained with our DummyClassifier
index += ["Dummy classifier"]

# Try different methods and hyper paramters Dummy classifier
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())
scores["f1_score"].append(f1_score(dummy_clf.predict(xtest), ytest,average='macro'))

df_scores = pd.DataFrame(scores, index=index)
df_scores

,Accuracy,Balanced accuracy,f1_score
Dummy classifier,0.931483,0.5,0.492308


In [15]:
# Try different methods and hyper paramters Dummy classifier vs LogisticRegression 
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=1000)

index += ["Logistic regression"]
cv_result = cross_validate(lr_clf, xtrain, ytrain, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())
lr_clf.fit(xtrain, ytrain)
scores["f1_score"].append(f1_score(lr_clf.predict(xtest), ytest,average='macro'))

df_scores = pd.DataFrame(scores, index=index)
df_scores

,Accuracy,Balanced accuracy,f1_score
Dummy classifier,0.931483,0.5,0.492308
Logistic regression,0.931483,0.5,0.492308


In [16]:
# Dummy classifier vs LogisticRegression vs Random forest
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=42, n_jobs=2)

index += ["Random forest"]
cv_result = cross_validate(rf_clf, X_new, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

scores["f1_score"].append(f1_score(rf_clf.fit(xtrain, ytrain).predict(xtest), ytest,average='macro'))

df_scores = pd.DataFrame(scores, index=index)
df_scores

,Accuracy,Balanced accuracy,f1_score
Dummy classifier,0.931483,0.500000,0.492308
Logistic regression,0.931483,0.500000,0.492308
Random forest,0.947269,0.597826,0.744845


In [17]:
# Dummy classifier vs LogisticRegression vs Random forest vs Logistic regression with balanced class weights
lr_clf.set_params(class_weight="balanced")

index += ["Logistic regression with balanced class weights"]
cv_result = cross_validate(lr_clf,  X_new, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

scores["f1_score"].append(f1_score(lr_clf.fit(xtrain, ytrain).predict(xtest), ytest,average='macro'))

df_scores = pd.DataFrame(scores, index=index)
df_scores

,Accuracy,Balanced accuracy,f1_score
Dummy classifier,0.931483,0.500000,0.492308
Logistic regression,0.931483,0.500000,0.492308
Random forest,0.947269,0.597826,0.744845
Logistic regression with balanced class weights,0.685694,0.661209,0.484808


In [18]:
# Dummy classifier vs LogisticRegression vs Random forest vs Logistic regression with balanced class weights vs Random forest with balanced class weights
rf_clf.set_params(class_weight="balanced")

index += ["Random forest with balanced class weights"]
cv_result = cross_validate(rf_clf, X_new, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

scores["f1_score"].append(f1_score(rf_clf.fit(xtrain, ytrain).predict(xtest), ytest,average='macro'))

df_scores = pd.DataFrame(scores, index=index)
df_scores

,Accuracy,Balanced accuracy,f1_score
Dummy classifier,0.931483,0.500000,0.492308
Logistic regression,0.931483,0.500000,0.492308
Random forest,0.947269,0.597826,0.744845
Logistic regression with balanced class weights,0.685694,0.661209,0.484808
Random forest with balanced class weights,0.947269,0.582246,0.492308


In [19]:
# Dummy classifier vs LogisticRegression vs Random forest vs Logistic regression with balanced class weights vs Random forest with balanced class weights vs Under-sampling + Logistic regression

from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

lr_clf = make_pipeline_with_sampler(
    RandomUnderSampler(random_state=42),
    LogisticRegression(max_iter=1000, class_weight="balance"),
)
index += ["Under-sampling + Logistic regression"]
cv_result = cross_validate(lr_clf, X_new, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

scores["f1_score"].append(f1_score(lr_clf.fit(xtrain, ytrain).predict(xtest), ytest,average='macro'))

df_scores = pd.DataFrame(scores, index=index)
df_scores

,Accuracy,Balanced accuracy,f1_score
Dummy classifier,0.931483,0.500000,0.492308
Logistic regression,0.931483,0.500000,0.492308
Random forest,0.947269,0.597826,0.744845
Logistic regression with balanced class weights,0.685694,0.661209,0.484808
Random forest with balanced class weights,0.947269,0.582246,0.492308
Under-sampling + Logistic regression,0.640981,0.652980,0.422500


In [20]:
# Dummy classifier vs LogisticRegression vs Random forest vs Logistic regression with balanced class weights vs Random forest with balanced class weights vs Under-sampling + Logistic regression vs Balanced random forest
from imblearn.ensemble import BalancedRandomForestClassifier

rf_clf = BalancedRandomForestClassifier(random_state=42, n_jobs=2)
index += ["Balanced random forest"]
cv_result = cross_validate(rf_clf, X_new, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

scores["f1_score"].append(f1_score(rf_clf.fit(xtrain, ytrain).predict(xtest), ytest,average='macro'))

df_scores = pd.DataFrame(scores, index=index)
df_scores

,Accuracy,Balanced accuracy,f1_score
Dummy classifier,0.931483,0.500000,0.492308
Logistic regression,0.931483,0.500000,0.492308
Random forest,0.947269,0.597826,0.744845
Logistic regression with balanced class weights,0.685694,0.661209,0.484808
Random forest with balanced class weights,0.947269,0.582246,0.492308
Under-sampling + Logistic regression,0.640981,0.652980,0.422500
Balanced random forest,0.647289,0.671961,0.422840
